In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import show_doc

# Directives

> The directives available in nbdev. 

Directives are special comments that are preceeded by `#|` that do pre or post processing of notebook data before docs are rendered. Some of these directives are part of Quarto, but others are unique to nbdev. All Quarto directives can be used in an nbdev notebook.

Directives can affect the following places:
* the Python module,
* the `__all__` export, see the [Python docs](https://docs.python.org/3/tutorial/modules.html#importing-from-a-package) for more,
* the tests,
* the documentation

## 📔 `#|default_exp <name>`

Required for every notebook. This directive names the module where things will be exported. For example:

```python
#| default_exp baz

# In a new notebook cell:

#| export
def my_function(): pass
```

If our package is named: `bitsnbytes` then we can do:

```python
from bitsnbytes.baz import my_function
```

You can define the package name by using `lib_name` in `settings.ini`.

## 📔 `#|export`

Exports the items in the cell into the generated module and documentation. For example:

```python
#|export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'
```

The documentation will look like this:

In [ ]:
#|exec_doc
#|echo: false
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
#|echo: false
# Note: we are using show_doc like this to simulate the effect of #|export without using export
show_doc(say_hello)

The docs are generated from this export using `showdoc.show_doc`. For a detailed discussion of `showdoc.show_doc` see [here](https://nbdev.fast.ai/explanations/showdoc.html). The exports are automatically included in `__all__` for the module.

## 📔 `#|export <some.thing>`

Similar to `#|export` above. Instead of exporting to the module named by `#|default_exp` export to the specified module.

If our package is named: `bitsnbytes`, and we have previously included: `#|default_exp core` in this notebook, and we have an existing notebook with `#|default_exp bar`, then:

```python
# Earlier in the notebook:

#|default_exp core

# A new notebook cell:

#|export bar
def foo(): pass
```

then we can import this as:
```python
from bitsnbytes.bar import foo
```

## 📔 `#|exporti`

An `i`nternal export. Not included in `__all__` or the docs. Useful for a function that is called by other functions in this module but is not part of the public API.

Equivalently you can prefix your function or method with `_` e.g. `def _private(): pass`.

## 📔 `#|exports`

A `s`ource export. Like `#|export` but in addition to showing docs via `showdoc.show_doc`, it also shows the source code. Given the following code:

```python
#|exports
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'
```

this will produce the following output:

In [ ]:
#|exec_doc
#|echo: true
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
#|echo: false
show_doc(say_hello)

## 📔 `#|exec_doc`

In [ ]:
#|hide
import datetime

Ensures that a cell is executed each time before generating docs. When a cell does not have this annotation, it is run according to the default rules [described here](https://nbdev.fast.ai/explanations/showdoc.html). 

In [ ]:
datetime.datetime.now()

datetime.datetime(2022, 8, 16, 21, 2, 42, 918107)

However with the annotation:
```python
#|exec_doc
datetime.datetime.now()
```

we can see that the time has been updated:

In [ ]:
#|exec_doc
datetime.datetime.now()

datetime.datetime(2022, 8, 16, 21, 3, 42, 444782)

## 📔 `#|notest`

In [ ]:
#|notest
assert 1 == 2

Do not run this cell during testing. This is the default test flag that [comes with nbdev](https://github.com/fastai/nbdev/blob/cdd1187d9d4d130197259143ad00bdeb9c4266d4/settings.ini#L51).

It is possible to specify additional values. For example see [fastai](https://github.com/fastai/fastai/blob/c5b9aa050e1ed382b40a7f772a07d74453fdcacb/settings.ini#L27). This allows you to toggle certain tests on and off by running `nbdev_test --flags`.

## 📔 `#|hide`

When you use this directive, you will not see the cell input or output. Given the following input:

```python
#|hide
print('you will not see this')
```

In [ ]:
#|hide
print('you will not see this')

you will not see this


You will not see any output above.

As opposed to:
```python
print('you will see this')
```

In [ ]:
print('you will see this')

you will see this


This is equivalent to the combination of:
```
#| echo: false
```
which allows you to hide the cell input and
```
#| output: false
```
which allows you to hide the cell output.

## 📔 `#|hide_line`

You can use this to hide as specific line in your code.  For example:

```python
def _secret(): ...

for i in range(3):
    _secret() #|hide_line
    print(i)
```

becomes this:

In [ ]:
def _secret(): ...

for i in range(3):
    _secret() #|hide_line
    print(i)

0
1
2


## 📔 `#|filter_stream <keyword> ...`

This allows you to filter lines containing specific keywords in cell outputs.  For example


```python
#|filter_stream FutureWarning MultiIndex
print('\n'.join(['A line', 'Foobar baz FutureWarning blah', 
                 'zig zagMultiIndex zoom', 'Another line.']))
```

becomes this:

In [ ]:
#|echo: false
#|filter_stream FutureWarning MultiIndex
print('\n'.join(['A line', 'Foobar baz FutureWarning blah', 
                 'zig zagMultiIndex zoom', 'Another line.']))

## 📔 `no directive`


For completeness sake, when a cell has no directive applied to it e.g. the cell does not include any line that starts with `#|`, then the cell:

* is show in the docs as-is,
* manually added `showdoc.show_doc` cells are run automatically,
* executes in tests,
* if it contains `import x` or `from x import y` then the cell is run

See [this page](https://nbdev.fast.ai/explanations/showdoc.html) for more.

# Quarto Directives

See the [Quarto docs](https://quarto.org/docs/computations/execution-options.html) for even more execution options.

## ❤️ `#|output: <true|false|asis>`

Shows or hides the output of evaluating the cell. For example:

```python
#|output: true
1 + 1
```
shows the result of the output:

In [ ]:
#|output: true
1 + 1

2

whereas:
```python
#|output: false
1 + 1
```
produces:

In [ ]:
#|output: false 
1 + 1

2

with no output.

Finally there is `asis` which is used for displaying raw markdown. The following cell will produce the raw output `### Heading 3` which will be rendered as a heading sized markdown element.

In [ ]:
#|output: asis
print("### Heading 3")

### Heading 3


In [ ]:
#|hide
from IPython.display import display, Markdown

Alternatively to display markdown you can use:
```python
#| echo: false
from IPython.display import display, Markdown
display(Markdown("### Heading 3\n"))
display(Markdown("#### Heading 4\n"))
```

In [ ]:
#|echo: false
display(Markdown("### Heading 3\n"))
display(Markdown("#### Heading 4\n"))

### Heading 3


#### Heading 4


which results in actual markdown heading being rendered as above.

## ❤️ `#|echo: <true|false>`

This makes sure that only the output of a code cell is shown, not its input.

```python
#|echo: false
print('you can see the output but not the code!')
```

which results in:

In [ ]:
#|echo: false
print('you can see the output but not the code!')

you can see the output but not the code!


Compared to:
```python
#|echo: true
print('you can see the code and the output!')
```
which results in:

In [ ]:
#|echo: true
print('you can see the code and the output!')

you can see the code and the output!


## ❤️ `#|code-fold: <show|true>`

The `#|code-fold` directive allows you to collapse code (note: must use  `code-fold` and not `code_fold`):

```python
#|code-fold: true
def foo():
    print("this")
    print("is")
    print("a")
    print("long")
    print("body")
```

which results in (click to expand):

In [ ]:
#|code-fold: true
def foo():
    print("this")
    print("is")
    print("a")
    print("long")
    print("body")

With `true` it is collapsed by default, with `show` it is shown by default and still collapsible. The directive also works with `#|exports` but not with `#|export`:

```python
#|exports
#|code-fold: true
def foo():
    print("this")
    print("is")
    print("a")
    print("long")
    print("body")
```

which results in (click to expand):

In [ ]:
#|echo: true
#|code-fold: true
def foo():
    print("this")
    print("is")
    print("a")
    print("long")
    print("body")

## ❤️ `#|eval: <true|false>`

This quarto directive is respected by `nbdev` to mean that you do not want to execute this cell during testing. The `#|notest` directive gives you more fine-grained control over testing than this.

Example:

```python
#|eval: false
raise Exception("I'm not raised because I'm not run")
```

In [ ]:
#|eval: false
raise Exception("I'm not raised because I'm not run")

Compared to:

```python
#|eval: true
assert 1 + 1 == 2
```

In [ ]:
#|eval: true
assert 1 + 1 == 2